In [227]:
import numpy as np
import copy
eps = 1e-6

In [402]:
def simplex_canonical_basis(c, A, b, x0, basis):
    n = A.shape[1]
    m = A.shape[0]
    inv = np.linalg.inv(A[:, basis])
    bb = inv.dot(b)
    As = inv.dot(np.delete(A, basis, axis=1))
    G = np.zeros((m + 1, n + 1))
    G[:m, 0] = bb
    G[m][0] = -c.dot(x0)
    d = np.delete(c, basis) - np.transpose(As).dot(c[basis])
    for i in range(m):
        k = 0
        for j in range(1, n + 1):
            if (basis[i] == j - 1):
                G[i][j] = 1
            if (not (j - 1 in basis)):
                G[i][j] = As[i][k]
                k += 1
    k = 0
    for j in range(1, n + 1):
        if (not (j - 1 in basis)):
            G[m][j] = d[k]
            k += 1
    basises = set()
    while (True):
        basises.add(tuple(basis))
        cont = False
        for j in range(1, n + 1):
            if (G[m][j] < eps or j - 1 in basis):
                continue
            ind = -1
            mn = float("inf")
            for i in range(m):
                if (G[i][j] > eps and G[i][0] / G[i][j] < mn):
                    ind = i
                    mn = G[i][0] / G[i][j]
            if (ind == -1):
                continue
            cont = True
            basis[ind] = j - 1
            basises.clear()
            G[ind] = G[ind] / G[ind][j]
            for i in range(m + 1):
                if (i == ind):
                    continue
                G[i] -= G[i][j] * G[ind]
            break
        if (cont):
            continue
        for j in range(1, n + 1):
            if (G[m][j] < -eps or j - 1 in basis):
                continue
            ind = -1
            mn = float("inf")
            for i in range(m):
                if (G[i][j] > eps and G[i][0] / G[i][j] < mn):
                    mn = G[i][0] / G[i][j]
            for i in range(m):
                if (G[i][j] > eps and G[i][0] / G[i][j] < mn + eps):
                    new_basis = copy.copy(basis)
                    new_basis[i] = j - 1
                    if (not (tuple(new_basis) in basises)):
                        ind = i
                        break
            if (ind == -1):
                continue
            cont = True
            basis[ind] = j - 1
            G[ind] = G[ind] / G[ind][j]
            for i in range(m + 1):
                if (i == ind):
                    continue
                G[i] -= G[i][j] * G[ind]
            break
        if (not cont):
            break
    A = G[:m, 1:]
    b = G[:m, 0]
    sol = np.linalg.solve(A[:, basis], b)
    x = np.zeros(n)
    for i in range(m):
        x[basis[i]] = sol[i]
    return x, basis

In [403]:
def simplex_canonical(c, A, b):
    n = A.shape[1]
    m = A.shape[0]
    for i in range(m):
        if (b[i] < -eps):
            A[i] *= -1
            b[i] *= -1
    cc = np.hstack((np.zeros(n), -np.ones(m)))
    AA = np.hstack((A, np.eye(m)))
    x0 = np.hstack((np.zeros(n), b))
    basis = [n + i for i in range(m)]
    x0, basis = simplex_canonical_basis(cc, AA, b, x0, basis)
    x0 = x0[:n]
    x, basis = simplex_canonical_basis(c, A, b, x0, basis)
    return x

In [404]:
def simplex(c, opt='max', leq=None, eq=None, geq=None):
    if (geq is not None):
        geq[0] *= -1
        geq[1] *= -1
    if (leq is None):
        leq = geq
    else:
        if (geq is not None):
            leq[0] = np.vstack((leq[0], geq[0]))
            leq[1] = np.hstack((leq[1], geq[1]))
    n = 0
    m = 0
    k = 0
    if (eq is not None):
        n = eq[0].shape[1]
        k = eq[0].shape[0]
    else:
        n = leq[0].shape[1]
        m = leq[0].shape[0]
    if (eq is not None):
        eq[0] = np.hstack((eq[0], -eq[0]))
    if (leq is not None):
        leq[0] = np.hstack((leq[0], -leq[0]))
    c = np.hstack((c, -c))
    if (m > 0):
        leq[0] = np.hstack((leq[0], np.eye(m)))
        c = np.hstack((c, np.zeros(m)))
        if (eq is None):
            eq = leq
        else:
            eq[0] = np.hstack((eq[0], np.zeros((k, m))))
            eq[0] = np.vstack((eq[0], leq[0]))
            eq[1] = np.vstack((eq[1], leq[1]))
    if (opt == 'min'):
        c *= -1
    x = simplex_canonical(c, eq[0], eq[1])
    return x[:n] - x[n:2 * n]

In [405]:
c = np.array([6, 1, 4, -5])
A = np.array([
    [3, 1, -1, 1],
    [5, 1, 1, -1]
])
b = np.array([4, 4])
x0 = np.array([1, 0, 0, 1])
print(simplex_canonical_basis(c, A, b, x0, [0, 3]))

(array([0., 4., 0., 0.]), [1, 3])


In [406]:
c = np.array([1, 2, 3, -1])
A = np.array([
    [1, -3, -1, -2],
    [1, -1, 1, 0]
])
b = np.array([-4, 0])
x0 = np.array([0, 1, 1, 0])
print(simplex_canonical_basis(c, A, b, x0, [1, 2]))

(array([2., 2., 0., 0.]), [1, 0])


In [407]:
c = np.array([1, 2, 1, -3, 1])
A = np.array([
    [1, 1, 0, 2, 1],
    [1, 1, 1, 3, 2],
    [0, 1, 1, 2, 1]
])
b = np.array([5, 9, 6])
x0 = np.array([0, 0, 1, 2, 1])
print(simplex_canonical_basis(c, A, b, x0, [2, 3, 4]))

(array([3., 2., 4., 0., 0.]), [2, 1, 0])


In [408]:
c = np.array([1, 1, 1, -1, 1])
A = np.array([
    [1, 1, 2, 0, 0],
    [0, -2, -2, 1, -1],
    [1, -1, 6, 1, 1]
])
b = np.array([4, -6, 12])
x0 = np.array([1, 1, 2, 0, 0])
print(simplex_canonical_basis(c, A, b, x0, [0, 1, 2]))

(array([4., 0., 0., 1., 7.]), [0, 4, 3])


In [409]:
c = np.array([1, -4, 3, -10])
A = np.array([
    [1, 1, -1, -10],
    [1, 14, 10, -10]
])
b = np.array([0, 11])
print(simplex_canonical(c, A, b))

[1. 0. 1. 0.]


In [414]:
c = np.array([-1, 5, 1, -1])
l = np.array([
    [1, 3, 3, 1],
    [2, 0, 3, -1]
])
bl = np.array([3, 4])
g = np.eye(4)
bg = np.zeros(4)
print(simplex(c, opt='min', leq=[l, bl], geq=[g, bg]))

[0. 0. 0. 3.]


In [411]:
c = np.array([1, 1, -1, 1, -2])
A = np.array([
    [3, 1, 1, 1, -2],
    [6, 1, 2, 3, -4],
    [10, 1, 3, 6, -7]
])
b = np.array([10, 20, 30])
print(simplex_canonical(c, A, b))

[ 0.  0. 10.  0.  0.]


In [415]:
c = np.array([-1, -1, 2, 3])
A = np.array([
    [2, -1, 1, 0],
    [-1, 2, 0, 1]
])
b = np.array([1, 1])
x0 = np.array([0, 0, 1, 1])
print(simplex_canonical_basis(c, A, b, x0, [2, 3]))

(array([0., 0., 1., 1.]), [2, 3])
